In [244]:
# import modules
import psycopg2
import pandas as pd
import re
import pymystem3
from pymystem3 import Mystem
from tqdm import tqdm
import nltk
nltk.download('stopwords')
import json
from nltk.corpus import stopwords
import gensim
from gensim.models.word2vec import Word2Vec
import numpy as np

[nltk_data] Downloading package stopwords to C:\Users\Pavel The
[nltk_data]     I\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [245]:
# getting data
conn = psycopg2.connect(
    host='stepik-bd.c2hf8kenf8dw.us-east-1.rds.amazonaws.com',
    user='stepik_master',
    password='Stepikpa$$word',
    database='stepik_api')

query = "SELECT id, title FROM courses"
data = pd.read_sql(query, con=conn)
conn.close()

In [246]:
# remove the garbage
data.title = data.title.apply(lambda x: " ".join(re.sub("[^A-Za-z0-9а-яА-Я+#ёЁйЙ -]+", " ", re.sub("<(.*?)>", "", x.lower())).split()))

In [247]:
# lemmatizing titles
def lemmatize_words(string):
    m = Mystem()
    return ' '.join(m.lemmatize(string))

data['text_lem'] = None
for ind in tqdm(data.index):
    data.loc[ind, 'text_lem'] = lemmatize_words(data.loc[ind, 'title'])

100%|██████████████████████████████████████████████████████████████████████████████| 1110/1110 [15:50<00:00,  1.17it/s]


In [249]:
#data = pd.read_csv('data_lem_title_full.csv')
data.to_csv('data_lem_title_full.csv', index=False)

In [250]:
# killing stop-words
def kill_stop_words(string):
    with open('stopwords-ru.json', 'r', encoding='utf-8') as read_file:
        stop_words = json.loads(read_file.read())
    
    stop_words.extend(["will","основной","весь","мочь", "type","types","тип", "can", "com", "это","тест","наш","каждый","курс","http","https","язык","org","свой","задача","course","также","урок","работа","тема","stepik","модуль","задание","который", '\n', "язык"])
    stop_words.extend(list(stopwords.words('english')))
    anti_stop = [wrd for wrd in str(string).split() if wrd not in stop_words]
    return ' '.join(anti_stop)

data['title_lem'] = data.text_lem.apply(kill_stop_words)

In [251]:
data.title_lem = data.title_lem.apply(lambda x: " ".join(re.sub("[^A-Za-z3а-яА-Я+#ёЁйЙ -]+", " ", re.sub("<(.*?)>", "", x.lower())).split()))

In [252]:
#model
import fasttext
import fasttext.util
ft = fasttext.load_model('cc.ru.300.bin')

In [253]:
def text_to_vector(string):
    list_wrds = string.split()
    var = list()
    for wrd in list_wrds:
        var.append(ft.get_word_vector(wrd))
    return np.mean(np.array(var), axis=0)

In [254]:
title_vecs = data.title_lem.apply(text_to_vector)

In [266]:
data_vecs = pd.DataFrame(data['id']).join(title_vecs).dropna()
data_vecs.columns = ['id', 'vecs']

In [269]:
data[['id', 'title_lem', 'title']]

,id,title_lem,title
0,71155,обучение наставник,обучение наставников
1,70810,формула образование школа ученик родитель учитель,формула образования школа ученик родитель учитель
2,70727,егэ нормальный вариант,егэ по русскому нормальные варианты
3,70710,свободный погружение субд,свободное погружение в субд
4,70581,набирать адаптировать команда удаленщик отдел ...,как набрать и адаптировать команду удаленщиков...
...,...,...,...
1105,71439,машинный обучение управление проект преподаватель,машинное обучение и управление проектами в it ...
1106,68789,алгоритмика класс,алгоритмика 5 класс
1107,67676,биология класс,биология 7 класс
1108,58873,основа компьютерный сеть,основы компьютерных сетей


In [277]:
data_vecs = data_vecs.merge(data[['id', 'title_lem', 'title']], on='id')

In [377]:
keys = {
    "программирование": ['python', 'java', 'javascript', 'kotlin', 'scala', 'c++', 'git', 'linux', 'программирование', 'php', 'web', 'it', 'ai', 'субд', 'r', 'алгоритм', "дискретный", "computer", "programming", 'scala', 'sql', "deep", "ironpython", "android", "ios", "swift", "многопоточность", "unix", "codewars", "разработка", "c#", "game", "software"],
    "web": ["html", "css", "javascript", "web", "php", "unix", "frontend", "интернет", "flask", "django", "веб"],
    "бд": ["sql", "субд", "hadoop", "данные", "vr"],
    "3d": ["3d", "3д", "компас", "blender", "трехмерный", "unity"],
    "статистика": ["машинный", "статистика", "r", "вероятность", "матиматический", "данные", "data", "нейронный", "excel", "deep", "tableu", "распределение", "probability"],
    "математика": ["дискретный", "математика", "алгоритм", "статистика", "математический", "анализ", "машинный", "data", "математика", "нейронный", "алгебра", "геометрия", "матанализ", "algebra"],
    "еств": ["химия", "биология", "физика", "анатомия", "bioinformatics", "биоинформатика", "chemistry", "physics", "химический", "астрономия"],
    "команда": ['команда', "проект", "наставник", "smm", "jira", "skills", "спикер", "agile"], 
    "начинающий": ["введение", "основа", "introduction", "базовый", "начало", "beginner", "начинающий"],
    "языки": ["русский", "английский", "english", "немецкий", "польский", "герб", "страна", "древнегреческий", "китайский", "ielts", "чешский", "японский", "французский", "russian", "language"],
    "школа": ["класс", "егэ", "огэ", "школьный", "кл", "география", "fran", "pascal", "школьник", "обществознание"],
    "пдд": ['пдд', "реанимация"],
    "экономика": ["маркетинг", "экономика", "excel", "инвестор", "инвестиция", "менеджер", "проект", "менеджмент", "бизнесс", "реклама", "опцион", "skills", "business", "предпринимательство", "торговля", "финансовый", "экономический", "трейдинг", "микроэкономика", "рынок", "маркетинговый", "management", "стартап"],
    "политика": ["политика", "история", "politics", "логика", "психология", "history", "право", "правоведение", "юриспрунденция"]
}

In [378]:
X = np.array([x for x in data_vecs.vecs.to_numpy()])
X = pd.DataFrame(X)
X["программирование"] = 0
X["web"] = 0
X["бд"] = 0
X["3d"] = 0
X["статистика"] = 0
X["математика"] = 0
X["еств"] = 0
X["команда"] = 0
X["начинающий"] = 0
X["языки"] = 0
X["школа"] = 0
X["пдд"] = 0
X["экономика"] = 0
X["политика"] = 0

In [379]:
for i in tqdm(range(data_vecs.shape[0])):
    text = data_vecs.loc[i, 'title_lem'].replace("-", "").split()
    for k in keys.keys():
        for val in keys[k]:
            if val in text:
                X.loc[i, k] = 1
            else:
                continue

100%|████████████████████████████████████████████████████████████████████████████| 1110/1110 [00:00<00:00, 2720.59it/s]


In [380]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=15, random_state=0)
clust = km.fit(X).predict(X)
data_vecs['cluster'] = clust

In [381]:
data_vecs.groupby('cluster').agg({"id": "count"})

,id
cluster,
0,54
1,43
2,110
3,34
4,41
5,411
6,54
7,49
8,47


In [382]:
max_cluster = np.argmax(np.array(data_vecs.groupby('cluster').agg({"id": "count"}).id))
sub_cl = data_vecs[data_vecs.cluster == max_cluster]

In [383]:
for i in tqdm(range(data_vecs.shape[0])):
    text = data_vecs.loc[i, 'title_lem'].replace("-", "").split()
    for k in keys.keys():
        for val in keys[k]:
            if val in text:
                X.loc[i, k] = 1
            else:
                continue

100%|████████████████████████████████████████████████████████████████████████████| 1110/1110 [00:00<00:00, 2713.95it/s]


In [402]:
X3 = np.array([x for x in sub_cl.vecs.to_numpy()])

In [385]:
for i in tqdm(range(sub_cl.shape[0])):
    text = data_vecs.reset_index().loc[i, 'title_lem'].replace("-", "").split()
    for k in keys.keys():
        for val in keys[k]:
            if val in text:
                X3.loc[i, k] = 1
            else:
                continue

100%|██████████████████████████████████████████████████████████████████████████████| 411/411 [00:00<00:00, 1132.13it/s]


In [403]:
X3

array([[ 0.04057591, -0.00155093, -0.00263742, ...,  0.01729428,
        -0.02432947,  0.01335699],
       [ 0.05170152,  0.01352578, -0.03196913, ...,  0.01677814,
         0.02140688,  0.00283054],
       [ 0.04890461, -0.03264511, -0.00181525, ..., -0.00886867,
         0.00532931, -0.01510109],
       ...,
       [ 0.02903979, -0.04898526, -0.01487778, ..., -0.03519501,
        -0.01512979,  0.00880668],
       [ 0.00719093, -0.03264761, -0.03265078, ..., -0.06324895,
        -0.01303423, -0.00493216],
       [ 0.0098462 ,  0.01249994, -0.00127916, ..., -0.01587147,
         0.00810701, -0.01928599]], dtype=float32)

In [423]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=8, random_state=0)
clust = km.fit(X3).predict(X3)
clust = [x + 14 if x != 0 else max_cluster for x in clust]
sub_cl['cluster'] = clust 

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [425]:
data_vecs.merge(sub_cl[['id', 'cluster']], on='id', how='left')

,id,vecs,title_lem,title,cluster_x,cluster_y
0,71155,"[0.040575914, -0.0015509333, -0.0026374192, 0....",обучение наставник,обучение наставников,5,21.0
1,70810,"[0.051701516, 0.01352578, -0.03196913, -0.0175...",формула образование школа ученик родитель учитель,формула образования школа ученик родитель учитель,5,15.0
2,70727,"[-0.008824989, 0.009678719, -0.035922557, -0.1...",егэ нормальный вариант,егэ по русскому нормальные варианты,4,NaN
3,70710,"[0.021721235, -0.028371545, 0.040372066, -0.01...",свободный погружение субд,свободное погружение в субд,10,NaN
4,70581,"[0.048904613, -0.03264511, -0.0018152483, 0.02...",набирать адаптировать команда удаленщик отдел ...,как набрать и адаптировать команду удаленщиков...,5,16.0
...,...,...,...,...,...,...
1105,71439,"[0.0098860655, -0.024736546, -0.0103480425, 0....",машинный обучение управление проект преподаватель,машинное обучение и управление проектами в it ...,9,NaN
1106,68789,"[0.0043830727, -0.010762613, -0.038223244, 0.0...",алгоритмика класс,алгоритмика 5 класс,6,NaN
1107,67676,"[0.027963763, -0.006683477, -0.055727765, 0.00...",биология класс,биология 7 класс,6,NaN
1108,58873,"[0.048682958, -0.10210681, 0.038794607, -0.002...",основа компьютерный сеть,основы компьютерных сетей,2,NaN


In [430]:
max_cluster

5

In [438]:
new = data_vecs.merge(sub_cl[['id', 'cluster']], on='id', how='left')
for i in range(new.shape[0]):
    #print(new.loc[i, "cluster_x"])
    if new.loc[i, "cluster_x"] == max_cluster:
        new.loc[i, "cluster_x"] = int(new.loc[i, "cluster_y"])
        #print(new.loc[i, "cluster_y"])

In [441]:
new[['id', 'title', 'cluster_x']].to_csv("clusters.csv")